# Welkom bij Tutorial 4

Dit is alweer de laatste tutorial van Algoritmiek en Datastructuren, waarin wij **grafen** gaan bestuderen. Een graaf is een datastructuur waarin nodes (die in de context van grafen ook wel **vertices** heten) aan elkaar verbonden zijn door middel van **edges**. 

De manier waarop er meestal over grafen geschreven wordt is al een tuple $G = (V, E)$ waarbij G de graaf is, V de *set* van nodes/vertices en E de *set* van edges. Ik schrijf hier *set* schuingedrukt, omdat dit een bekende datastructuur voor ons is: een container waarin geen dubbelen mogen zitten. 

Vertices zijn uiteraard een classes die data kunnen bevatten, maar ook de edges kunnen soms data bevatten. Bijvoorbeeld, in een shortest path problem (zie Reader) stellen de vertices/nodes locaties voor, en bevatten dus de naam van de locatie, en de edges verbindingen tussen deze locaties. De edges bevatten de *afstand* (bijvoorbeeld in kilometers) tussen deze locaties. 

Grafen zijn een zeer flexibele manier van data structuren: zolang er maar verbindingen tussen nodes en edges gelegd kunnen worden is het een valide implementatie van een graaf. Dit kan je natuurlijk op heel veel verschillende manieren doen. Laten we daar eens een paar van gaan bekijken. 

We gaan dit graafje maken: https://en.wikipedia.org/wiki/File:Dijkstra_Animation.gif 

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import time
import statistics as stats
        

In [2]:
# Een zeer generieke manier om een graaf de implementeren is er 
# daarwerkelijk twee sets van te maken op basis van twee classes: 
class Vertex: 
    def __init__(self, identifier, data_, distance_ = 10000000, prev_ = None, solved_ = False ):
        self.id = identifier
        self.data = data_
        self.distance = distance_
        self.prev = prev_
        self.solved = solved_
        
    def __eq__(self, other): #nodig om aan een set toe te voegen
        return self.id == other.id
    
    def __hash__(self): #nodig om aan een set toe te voegen
        return hash(self.id)
    
    def __repr__(self):
        return str(self.id)+":"+str(self.data)
        
class Edge:
    def __init__(self, vertex1, vertex2, data_):
        if(vertex1.id<vertex2.id):
            self.v1 = vertex1
            self.v2 = vertex2
        else:
            self.v1 = vertex2
            self.v2 = vertex1
        self.data = data_
        
    def __eq__(self, other): #nodig om aan een set toe te voegen
        return self.v1.id == other.v1.id and self.v2.id == self.v2.id
    
    def __hash__(self): #nodig om aan een set toe te voegen
        return hash(str(self.v1.id)+","+str(self.v2.id))
    
    def __repr__(self):
        return "("+str(self.v1.id)+","+str(self.v2.id)+"):"+str(self.data)

class CGraph:
    def __init__(self):
        self.V = set()
        self.E = set()
        
    def __str__(self):
        return "V: "+str(self.V)+"\nE: "+str(self.E)
            
        
# So, for a simple shortest path problem: 
gr = CGraph()
v1 = Vertex(1,"start",distance_ = 0 )
v2 = Vertex(2,"")
v3 = Vertex(3,"")
v4 = Vertex(4,"")
v5 = Vertex(5,"")
v6 = Vertex(6,"goal")
gr.V.add(v1)
gr.V.add(v2)
gr.V.add(v3)
gr.V.add(v4)
gr.V.add(v5)
gr.V.add(v6)
e1 = Edge(v1,v2,4)
e2 = Edge(v1,v3,2)
e3 = Edge(v2,v3,5)
e4 = Edge(v2,v4,10)
e5 = Edge(v3,v5,3)
e6 = Edge(v4,v6,11)
e7 = Edge(v5,v4,4)
gr.E.add(e1)
gr.E.add(e2)
gr.E.add(e3)
gr.E.add(e4)
gr.E.add(e5)
gr.E.add(e6)
gr.E.add(e7)
print(gr)

# Is dit eigenlijk al best een langdradige manier van doen...

V: {1:start, 2:, 3:, 4:, 5:, 6:goal}
E: {(2,4):10, (4,5):4, (1,2):4, (3,5):3, (2,3):5, (1,3):2, (4,6):11}


In [3]:
#Daarom kiezen mensen er vaak voor om een graaf makkelijker te respresenteren met standaard datatypes
#bijvoorbeeld: als we een dictionary maken met node identifiers als keys, en een tuple van (data, edges) als values
#met edges als een dictionary met vertex ids van de verbonden vertices als keys en de data van de edge als key
#krijgen we zoiets voor dezelfde graaf
DGraph = dict
gr2 = {1: ("start", {2:7, 3:9, 6:14}),
       2: ("", {1:7, 3:10, 4:15}), 
       3: ("", {1:9, 2:10, 4:11, 6:2}), 
       4: ("", {2:15, 3:11, 5:6}), 
       5: ("goal", {4:6, 6:9}), 
       6: ("", {1:14, 3:2, 5:9})
      }
print(gr2)
#Dat is makkelijker in te voeren... Maar je moet wel goed bijhouden wat ookalweer wat is. 
#Bovendien heb ik er nu wat redundantie ingezet (elke node bevat de edges waaraan het verbonden zit, dus elke 
# edge zit twee maal in deze datastructuur)

{1: ('start', {2: 7, 3: 9, 6: 14}), 2: ('', {1: 7, 3: 10, 4: 15}), 3: ('', {1: 9, 2: 10, 4: 11, 6: 2}), 4: ('', {2: 15, 3: 11, 5: 6}), 5: ('goal', {4: 6, 6: 9}), 6: ('', {1: 14, 3: 2, 5: 9})}


## Opgave 1: Dijkstra

Laten we eens gaan kijken hoe goed de twee mogelijke implementaties van een graaf hierboven zijn (dus die met de expliciete klasses voor Vertices en Edges, en die die de hele graaf in één dictionary gooit. 

Schrijf implementaties van Dijkstra's algoritme voor de beide implementaties van grafen (CGraph en DGraph). Vergelijk de snelheid van beide versies van Dijkstra's algoritme door beiden meerdere keren te runnen. Vergeet niet de standaardafwijkingen te berekenen. Is één implementatie beter dan de andere? 

In [2]:
#Schrijf hier de code voor opgave 1
class Vertex: 
    def __init__(self, identifier, data_, distance_ = 10000000, prev_ = None, solved_ = False ):
        self.id = identifier
        self.data = data_
        self.distance = distance_
        self.prev = prev_
        self.solved = solved_
        
    def __eq__(self, other): #nodig om aan een set toe te voegen
        return self.id == other.id
    
    def __hash__(self): #nodig om aan een set toe te voegen
        return hash(self.id)
    
    def __repr__(self):
        return str(self.id)+":"+str(self.data)
        
class Edge:
    def __init__(self, vertex1, vertex2, data_):
        if(vertex1.id<vertex2.id):
            self.v1 = vertex1
            self.v2 = vertex2
        else:
            self.v1 = vertex2
            self.v2 = vertex1
        self.data = data_
        
    def __eq__(self, other): #nodig om aan een set toe te voegen
        return self.v1.id == other.v1.id and self.v2.id == self.v2.id
    
    def __hash__(self): #nodig om aan een set toe te voegen
        return hash(str(self.v1.id)+","+str(self.v2.id))
    
    def __repr__(self):
        return "("+str(self.v1.id)+","+str(self.v2.id)+"):"+str(self.data)

class CGraph:
    def __init__(self):
        self.V = set()
        self.E = set()
        
    def __str__(self):
        return "V: "+str(self.V)+"\nE: "+str(self.E)
            
        
# So, for a simple shortest path problem: 
gr = CGraph()
v1 = Vertex(1,"start",distance_ = 0 )
v2 = Vertex(2,"")
v3 = Vertex(3,"")
v4 = Vertex(4,"")
v5 = Vertex(5,"")
v6 = Vertex(6,"goal")
gr.V.add(v1)
gr.V.add(v2)
gr.V.add(v3)
gr.V.add(v4)
gr.V.add(v5)
gr.V.add(v6)
e1 = Edge(v1,v2,4)
e2 = Edge(v1,v3,2)
e3 = Edge(v2,v3,5)
e4 = Edge(v2,v4,10)
e5 = Edge(v3,v5,3)
e6 = Edge(v4,v6,11)
e7 = Edge(v5,v4,4)
gr.E.add(e1)
gr.E.add(e2)
gr.E.add(e3)
gr.E.add(e4)
gr.E.add(e5)
gr.E.add(e6)
gr.E.add(e7)

def shortest_path_func(end, shortest_path_tmp=[]):
    if len(shortest_path_tmp) == 0:
        shortest_path_tmp.append(end)
    elif(shortest_path_tmp[-1].data == "start"):
        return shortest_path_tmp
    shortest_path_tmp.append(shortest_path_tmp[-1].prev)
    return shortest_path_func(end, shortest_path_tmp )
        

def dijkstra(graph, start, end):
    solved = []
    queue = [start]

    while queue:
        min_distance = 10000000
        min_distance_index = 0

        for i in range(len(queue)):
            if queue[i].distance < min_distance:
                min_distance_index = i #set the index of the shortest distance to the index of the current vertex
                min_distance = queue[i].distance #set the shortest distance to the distance of the current vertex
            
        next_node = queue[min_distance_index]
        solved.append(next_node)
        queue.remove(next_node)
        next_node.solved = True

        for i in graph.E:
            if i.v1 == next_node and i.v2.solved == False:
                if i.v2 not in queue:
                    queue.append(i.v2)
                current_distance = next_node.distance + i.data
                if i.v2.distance > current_distance:
                    i.v2.distance = current_distance
                    i.v2.prev = next_node
            
            if i.v2 == next_node and i.v1.solved == False:
                if i.v1 not in queue:
                    queue.append(i.v1)
                current_distance = next_node.distance + i.data
                if i.v1.distance > current_distance:
                    i.v1.distance = current_distance
                    i.v1.prev = next_node

    shortest_path = shortest_path_func(end)
    shortest_path.reverse()

    return shortest_path, solved[-1].distance #return the shortest path and the distance of the last vertex in the shortest path

print(dijkstra(gr, v1, v6))

        
            


            



([1:start, 3:, 5:, 4:, 6:goal], 20)


*Schrijf hier de tekst-antwoorden voor Opgave 1*

## Opgave 2: Een andere implentatie?

Verzin (evt. samen met je teamleden) nog een andere manier om grafen te implementeren, en schrijf vervolgens weer een bijbehorende implementatie van Dijkstra's algoritme. Is deze beter dan de vorige twee manieren? Toon dit wederom aan door metingen te doen. 

Ter inspiratie: kijk eens naar **adjacency lists** https://en.wikipedia.org/wiki/Adjacency_list en **adjacency matrices** https://en.wikipedia.org/wiki/Adjacency_matrix . Zou je de edges misschien (evt. per vertex) kunnen sorteren op een manier dat Dijkstra's algoritme makkelijker maakt, en zo ja, is dat computationeel de moeite waard?

In [ ]:
#Schrijf hier de code voor opgave 2

*Schrijf hier de tekst-antwoorden voor Opgave 2*

## Ten Slotte

Grafen kun je dus op heel veel verschillende manieren implementeren. Sommige manieren zijn beter dan anderen voor specifieke algoritmes. Let dus altijd goed op hoe je een graaf implementeert als je aan verschillende toepassingen denkt. 

Ook verschilt de implementatie uiteraard als er wel of geen data in vertices en/of edges moeten worden opgeslagen. 

In het tweede grote practicum, zullen we kijken naar een coordination graph. Dit is een graaf waarbij elke vertex een beslisvariabele voorstelt met een beperkt aantal mogelijke beslissingen (acties / waarden). Elke edge representeert welke lokale beloning je krijgt voor de mogelijke beslissingen voor de beslisvariabelen die dezen verbindt. Kijk bij dit practicum eerst eens goed hoe de graaf geïmplementeerd is. Mocht je daar vragen over hebben stel ze dan z.s.m. aan een van de docenten of aan de studentassistent.   